In [ ]:
import keras
import csv
import os
import numpy as np
from keras.layers import *
from keras import optimizers
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.applications.inception_v3 import InceptionV3
from keras import backend as k
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model

In [ ]:
ROOT_DIR = os.path.normpath(os.path.join(os.path.dirname(os.path.realpath('__file__'))))
TRAIN_DIR = os.path.join(ROOT_DIR, 'domain_adaptation_images', 'amazon', 'images')
VAL_DIR = os.path.join(ROOT_DIR, 'domain_adaptation_images', 'dslr', 'images')
TEST_DIR = os.path.join(ROOT_DIR, 'domain_adaptation_images', 'webcam', 'images')

In [ ]:
img_width, img_height = 520, 520

In [ ]:
batch_size = 8
epoch = 10
drop_prob = 0.5
out_class = 31

In [ ]:
train_gen = ImageDataGenerator(rescale=1. / 255, zoom_range=0.2, horizontal_flip = True, vertical_flip = True)

In [ ]:
test_gen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
train_generator = train_gen.flow_from_directory(TRAIN_DIR, target_size=(img_width, img_height), 
                                                batch_size=batch_size, class_mode='categorical')

In [ ]:
val_generator = test_gen.flow_from_directory(VAL_DIR, target_size=(img_width, img_height), 
                                                batch_size=batch_size, class_mode='categorical')

In [ ]:
test_generator = test_gen.flow_from_directory(TEST_DIR, target_size=(img_width, img_height), 
                                                batch_size=batch_size, class_mode='categorical')

In [ ]:
input_ = Input(shape=(img_width, img_height, 3))
transfer_model = InceptionV3(input_tensor = input_, weights = 'imagenet', include_top = False)

In [ ]:
x = transfer_model.output

x = MaxPooling2D()(x)

x = Dropout(drop_prob)(x)

out = Dense(out_class, activation = 'softmax')(x)

In [ ]:
model = Model(inputs = transfer_model.input, outputs = out)

In [ ]:
adam_ = optimizers.Adam(lr = 1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-8)

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = adam_, metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
for layer in transfer_model.layers:
    layer.trainable = False

In [ ]:
tensorboard = TensorBoard(log_dir='./DA_logs/', histogram_freq = 1, write_graph=True, write_images = True)

In [ ]:
checkpointer = ModelCheckpoint(filepath='checkpoint_DA.h5', save_best_only=True)

In [ ]:
callback_list = [tensorboard, checkpointer]

In [ ]:
model.fit_generator(train_generator, steps_per_epoch = int(2817/batch_size), epochs = epoch, shuffle = True, 
                   validation_data = val_generator, validation_steps = int(498/batch_size), callbacks = callback_list)